In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
import requests
import os
from glob import glob
from pprint import pprint

from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext
)
from llama_index.core import SummaryIndex, Settings
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.anthropic import Anthropic
from llama_index.core.node_parser import SentenceSplitter
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.objects import ObjectIndex
from llama_index.packs.agents_coa import CoAAgentPack
from llama_index.tools.code_interpreter.base import CodeInterpreterToolSpec# from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.agent import ReActAgent, StructuredPlannerAgent, FunctionCallingAgentWorker

from Pycrafter6500.pycrafter6500 import dmd

In [28]:
dlpc9000_docs_dirpath = "../../manuals/dmd"

In [37]:
dmd_docs_paths = {
    os.path.basename(pdf_doc).split(".")[0]: pdf_doc 
    for pdf_doc in glob(os.path.join(dlpc9000_docs_dirpath, "*.pdf"))
}

In [58]:
# Settings.llm = OpenAI(temperature=0, model="gpt-4o-mini")
Settings.llm = Anthropic(model='claude-3-5-sonnet-20240620')
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

# set up parser
parser = LlamaParse(
    result_type="markdown",  # "markdown" and "text" are available
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="anthropic-sonnet-3.5"
)


In [119]:

doc_name = 'dlpu018j'
persist_dir_root = "./storage/dlpc9000"

if not os.path.exists(persist_dir_root):
    os.makedirs(persist_dir_root)
doc_persist_dir = os.path.join(persist_dir_root, doc_name)

print(f"Building agent for document {doc_name}")


node_parser = SentenceSplitter()
if not os.path.exists(doc_persist_dir):
    # build vector index
    file_extractor = {".pdf": parser, ".html": parser}
    doc = SimpleDirectoryReader(input_files=[dmd_docs_paths[doc_name]], file_extractor=file_extractor).load_data()
    # vector_index = VectorStoreIndex(doc)
    nodes = node_parser.get_nodes_from_documents(doc)
    vector_index = VectorStoreIndex(nodes)
    vector_index.storage_context.persist(
        persist_dir=doc_persist_dir
    )
else:
    vector_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir=doc_persist_dir),
    )

# build summary index
# summary_index = SummaryIndex(nodes)
# define query engines
vector_query_engine = vector_index.as_query_engine(llm=Settings.llm, similarity_top_k=10)
# summary_query_engine = summary_index.as_query_engine(llm=Settings.llm)

# define tools
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="vector_tool",
            description="This tool can query the DLPC900 (aka DLPC9000EVM) Programmer's Guide which contains information about communicating with the board and specifyihng.",
        ),
    ),
    # QueryEngineTool(
    #     query_engine=summary_query_engine,
    #     metadata=ToolMetadata(
    #         name="summary_tool",
    #         description=summary_query_engine.query("Summarize this document so a tool built upon it knows how to use it.").response,
    #     ),
    # ),
]

# build agent
function_llm = Anthropic(model="claude-3-5-sonnet-20240620")
agent = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=function_llm,
    verbose=True,
    # system_prompt=f"""\
    #             This tools has access to information about communicating with the DLPC900 (aka DLPC9000EVM) board over serial USB. \
    #             It can go through this document and understand how to format read and write commands to the board. \
    #             It can answer questions about how to format serial commands for different board commands. \
    #             It can can also answer questions going in the other direction, given a board command, it will provide the serial command. \
    #             Remember to always reference Section 1.2 when giving an answer since this explains how to format USB commands. \
    #             When you give an answer, please report what tables you used to get information. \
    #             Be careful, it also contains I2C commands in the same tables. \
    #             """,
)


# agents[doc_name] = agent
# query_engines[doc_name] = vector_index.as_query_engine(
#     similarity_top_k=2
# )

Building agent for document dlpu018j


In [120]:
agent = StructuredPlannerAgent(
    agent, tools=query_engine_tools + CodeInterpreterToolSpec().to_tool_list(), verbose=True
)

In [121]:
response = agent.query("What are three more general questions I should ask to understand how to answer the following question: How do I format USB commands to the device?").response

=== Initial plan ===
Query_DLPC900_Guide:
Search the DLPC900 (DLPC9000EVM) Programmer's Guide for information about USB communication and command formatting. -> Relevant sections or summaries from the guide about USB communication and command formatting.
deps: []


Analyze_USB_Communication_Info:
Analyze the information retrieved about USB communication with the DLPC900 board. -> Key points about USB communication with the device.
deps: ['Query_DLPC900_Guide']


Identify_Command_Formatting_Details:
Examine the information about command formatting for the DLPC900 board. -> Important aspects of command formatting for the device.
deps: ['Query_DLPC900_Guide']


Generate_General_Questions:
Based on the analyzed information about USB communication and command formatting, generate three general questions that would help in understanding how to format USB commands to the device. -> Three general questions that cover important aspects of formatting USB commands for the DLPC900 board.
deps: ['A

In [123]:
pprint(response)

('Certainly! Based on the analyzed information about USB communication and '
 'command formatting for the DLPC900 board, here are three general questions '
 'that would help in understanding how to format USB commands to the device:\n'
 '\n'
 '1. How should the 4-byte header of a USB command be structured, and what '
 'information does each byte contain?\n'
 '\n'
 '2. What is the significance of the Flag Byte in the USB command header, and '
 'how should it be set for different types of operations?\n'
 '\n'
 '3. How does the command structure differ between a read operation and a '
 'write operation when communicating over USB with the DLPC900 board?\n'
 '\n'
 'These questions focus on key aspects of USB command formatting for the '
 "DLPC900 board. Let's explore the answers to these questions based on the "
 "information we've analyzed:\n"
 '\n'
 '1. How should the 4-byte header of a USB command be structured, and what '
 'information does each byte contain?\n'
 '\n'
 'The 4-byte head

In [114]:
response_q1 = agent.query(f"Extract just the text fore the first proposed question from the following response: {response}")

=== Initial plan ===
Extract First Question:
Extract the text of the first proposed question from the given response using string manipulation. -> The extracted text of the first question.
deps: []


> Running step 174a8757-922c-4e71-b5dc-b31bdfc200ae. Step input: Extract the text of the first proposed question from the given response using string manipulation.
Added user message to memory: Extract the text of the first proposed question from the given response using string manipulation.
=== LLM Response ===
Certainly! To extract the text of the first proposed question from a given response using string manipulation, we'll need to use Python code. Let's use the code_interpreter function to accomplish this task. First, I'll write a Python script that demonstrates how to extract the first proposed question, and then we'll execute it using the code_interpreter.
=== Calling Function ===
Calling function: code_interpreter with args: {"code": "# Sample response text\nresponse = \"\"\"\nHere 

In [115]:
response_q1

Response(response='Great! The code has successfully extracted the first proposed question from the given response. Let\'s break down what the code does:\n\n1. It starts with a sample response text containing numbered questions.\n2. It finds the index of "1." to locate the start of the first question.\n3. It then searches for "2." to find the end of the first question.\n4. If "2." is not found (in case there\'s only one question), it extracts from the start index to the end of the string.\n5. It removes the number and any leading/trailing whitespace.\n6. Finally, it prints the extracted question.\n\nThe output shows that the code successfully extracted the first proposed question: "What is the capital of France?"\n\nThis code demonstrates how to use string manipulation to extract the first proposed question from a given response. You can modify the `response` variable in the code to work with any specific response text you have.\n\nIs there anything else you\'d like me to explain or dem

In [112]:
response = agent.query("How do I format USB commands to the device?").response
pprint(response)

No complex plan predicted. Defaulting to a single task plan.
=== Initial plan ===
default:
How do I format USB commands to the device? -> 
deps: []


> Running step c9e8fec9-e9df-46cf-8688-c986624da4b7. Step input: How do I format USB commands to the device?
Added user message to memory: How do I format USB commands to the device?
=== LLM Response ===
To answer your question about formatting USB commands for the DLPC900 (DLPC9000EVM) device, I'll need to consult the Programmer's Guide. Let me use the vector tool to get that information for you.
=== Calling Function ===
Calling function: vector_tool with args: {"input": "How are USB commands formatted for the DLPC900 device?"}
=== Function Output ===
USB commands for the DLPC900 device are formatted as variable length data packets with the following structure:

1. Report ID byte: Always set to 0x00 and is the first byte of all transfers.

2. Header: Consists of four bytes:
   - Flag byte: Contains information about the transaction type 

In [124]:
pack = CoAAgentPack(tools=query_engine_tools, llm=Settings.llm)

In [125]:
response = pack.run("How do I format USB commands to the device?").response
pprint(response)

==== Available Parsed Functions ====
def vector_tool(input: string):
   """This tool can query the DLPC900 (aka DLPC9000EVM) Programmer's Guide which contains information about communicating with the board and specifyihng."""
    ...
==== Generated Chain of Abstraction ====
Here's an abstract plan of reasoning for the given question:

1. First, we need to query the DLPC900 Programmer's Guide for information about USB commands.

[FUNC vector_tool("USB command format for DLPC900") = y1]

2. Review the information in y1 to understand the general format of USB commands for the device.

3. If y1 doesn't provide sufficient information about the USB command format, we may need to query for more specific details.

[FUNC vector_tool("DLPC900 USB command structure and syntax") = y2]

4. Analyze the information from y1 and y2 to identify key components of the USB command format, such as:
   - Command header (if any)
   - Command identifier or opcode
   - Parameter format
   - Data payload structu

In [126]:
pprint(response)

("Based on the previous reasoning, here's how to format USB commands for the "
 'DLPC900 device:\n'
 '\n'
 'To format USB commands for the DLPC900 device, follow these steps:\n'
 '\n'
 '1. Start with the Report ID byte: Always set this to 0x00.\n'
 '\n'
 "2. Add the Flag byte: This indicates whether it's a read or write operation "
 'and if a reply is needed.\n'
 '   - For write operations: Set bit 7 to 0\n'
 '   - For read operations: Set bit 7 to 1 and bit 6 to 1 (to request a '
 'reply)\n'
 '\n'
 '3. Include the Sequence byte: This can be used as a rolling counter, '
 'especially useful when expecting a response. You can increment this for each '
 'new command.\n'
 '\n'
 '4. Add the Length (2 bytes): This indicates the number of bytes in the '
 'payload.\n'
 '   - For write operations: Total bytes of command + data\n'
 '   - For read operations: Just the 2 command bytes\n'
 '\n'
 '5. Insert the USB Command (2 bytes): This represents the specific command '
 'you want to execute.\n'
 

In [68]:
response = agent.query("Write a Python method that takes a USB command and reads or writes it to the device.").response
pprint(response)

> Running step 44ce9c09-c5bc-4df6-8868-b64d7ffddd27. Step input: Write a Python method that takes a USB command and reads or writes it to the device.
Thought: The current language of the user is: English. To answer this question accurately, I need more information about the specific device and its communication protocol. However, I can provide a general template for a Python method that handles USB communication. I'll use the vector_tool to check if there's any relevant information about USB commands in the DLPC900 Programmer's Guide.
Action: vector_tool
Action Input: {'input': 'USB command communication DLPC900'}
Observation: The DLPC900 controller supports two host interface protocols for command communication: I²C and USB 1.1. While the context provides detailed information about the I²C interface, it does not give specific details about the USB communication protocol. The USB 1.1 interface is mentioned as an alternative to I²C for host-to-DLPC900 communication, but no further infor

In [99]:
response = agent.query("Write and validate a Python method that generates and returns the full command should I send over USB to set the device to idle. Include all parts of the command that need to be sent to the device as defined in section 1.2 including the header.").response
pprint(response)

=== Initial plan ===
Query_Documentation:
Query the vector_tool for information about the DLPC900 command structure, specifically for setting the device to idle and the header format. -> Information about the command structure, idle command, and header format for DLPC900.
deps: []


Write_Python_Method:
Write a Python method that generates the full command to set the DLPC900 device to idle, including the header. -> A Python method that constructs the full command.
deps: ['Query_Documentation']


Validate_Python_Method:
Use the code_interpreter to run and validate the Python method. -> Execution results and validation of the Python method.
deps: ['Write_Python_Method']


Finalize_and_Present:
Finalize the Python method based on validation results and present it to the user. -> The final, validated Python method that generates the full command to set the DLPC900 device to idle.
deps: ['Validate_Python_Method']


> Running step 779b69d8-a77b-44c5-ae5c-c2a9186d64fd. Step input: Query the v

In [100]:
def generate_idle_command(interface='USB'):
    """
    Generate the full command to set the DLPC900 device to idle mode.
    
    Args:
    interface (str): The interface being used, either 'USB' or 'I2C'. Defaults to 'USB'.
    
    Returns:
    bytes: The full command as a bytes object.
    
    Raises:
    ValueError: If an invalid interface is specified.
    """
    # Define headers
    USB_HEADER = b'\x02\x00'
    I2C_WRITE_HEADER = b'\x87'
    
    # Define the idle mode data byte
    IDLE_MODE = b'\x01'
    
    if interface.upper() == 'USB':
        header = USB_HEADER
    elif interface.upper() == 'I2C':
        header = I2C_WRITE_HEADER
    else:
        raise ValueError("Interface must be either 'USB' or 'I2C'")
    
    # Combine header and idle mode byte
    command = header + IDLE_MODE
    
    return command

# Test the function
usb_command = generate_idle_command()
print("USB command:", usb_command.hex())

USB command: 020001


In [44]:
dmd_device = dmd()

/Users/loyalshababo/dev/rtfm/Pycrafter6500/pycrafter6500.py:41: UserWarning: No DLP found. Simulating Device.
  warnings.warn("No DLP found. Simulating Device.")


In [110]:
dmd_device.idle_on()

[['01000000', '00000000', '00000011', '00000000', '00000001', '00000010', '00000001']]


In [111]:
dmd_device.changemode(3)

[['01000000', '00000000', '00000011', '00000000', '00011011', '00011010', '00000011']]


In [85]:
response = agent.query("What command should I send over USB to turn idle mode off?").response
pprint(response)

Added user message to memory: What command should I send over USB to turn idle mode off?
=== Calling Function ===
Calling function: vector_tool with args: {"input":"turn idle mode off"}
Got output: To turn off Idle Mode, any pattern sequences must first be stopped. Once the pattern sequences are stopped, Idle Mode can be disabled.

('To turn off Idle Mode, you need to follow these steps:\n'
 '\n'
 '1. Stop any pattern sequences that are currently running.\n'
 '2. Disable Idle Mode.\n'
 '\n'
 'Here are the commands you need to send over USB:\n'
 '\n'
 '1. **Stop Pattern Sequence:**\n'
 '   - Command: `0x1A34`\n'
 '   - Data: `0x00`\n'
 '\n'
 '2. **Disable Idle Mode:**\n'
 '   - Command: `0x1A35`\n'
 '   - Data: `0x00`\n'
 '\n'
 'Make sure to send these commands in the specified order to successfully turn '
 'off Idle Mode.')


In [86]:
response = agent.query("What command should I send over USB to turn idle mode on?").response
pprint(response)

Added user message to memory: What command should I send over USB to turn idle mode on?
=== Calling Function ===
Calling function: vector_tool with args: {"input":"turn idle mode on"}
Got output: To turn Idle Mode on, any pattern sequences must first be stopped. Once Idle Mode is enabled, the entire mirror array will continuously flip periodically between the on and off states with a 50/50 duty cycle pattern sequence. Additionally, when Idle Mode is enabled, the LED Enable outputs are disabled to prevent illumination on the DMD.

('To turn Idle Mode on, you need to ensure that any pattern sequences are '
 'stopped first. Once Idle Mode is enabled, the mirror array will continuously '
 'flip between the on and off states with a 50/50 duty cycle pattern sequence, '
 'and the LED Enable outputs will be disabled to prevent illumination on the '
 'DMD.\n'
 '\n'
 'Here is the command to send over USB to turn Idle Mode on:\n'
 '\n'
 '```\n'
 '0x1A 0x34 0x00\n'
 '```\n'
 '\n'
 'This command wi

In [ ]:
[['0x40', '0x0', '0x3', '0x0', '0x1', '0x2', '0x1']]

In [105]:
dmd_device = dmd()

/Users/loyalshababo/dev/rtfm/Pycrafter6500/pycrafter6500.py:41: UserWarning: No DLP found. Simulating Device.
  warnings.warn("No DLP found. Simulating Device.")


In [106]:
dmd_device.idle_off()

[[64, 0, 3, 0, 1, 2, 0]]


In [107]:
dmd_device.idle_on()

[[64, 0, 3, 0, 1, 2, 1]]


In [78]:
dlp_serial_cmd_interpreter_agent.state

AgentState(task_dict={})

In [73]:
dlp_serial_cmd_interpreter_agent.delete_task(task_id = dlp_serial_cmd_interpreter_agent.state.task_dict['2258a15c-dc74-40f2-8cd8-46e057c8ff1f'].task.task_id)

KeyError: '2258a15c-dc74-40f2-8cd8-46e057c8ff1f'

In [75]:
dlp_serial_cmd_interpreter_agent.statedd

AgentState(task_dict={})

In [80]:
response = dlp_serial_cmd_interpreter_agent.query("What command should I send over USB to set the device to idle?")

Added user message to memory: What command should I send over USB to set the device to idle?
=== Calling Function ===
Calling function: tool_dlpu018j with args: {"input":"Get the command to set the device to idle"}
Added user message to memory: Get the command to set the device to idle
=== Calling Function ===
Calling function: summary_tool with args: {"input":"command to set the device to idle"}
Got output: To set the device to idle, use the DMD Idle Mode command.

Got output: To set the device to idle, you should use the **DMD Idle Mode** command.



In [68]:
bytes = "0x1A 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x00".split(" ")

In [70]:
len(bytes)

101

In [42]:
response = dlp_serial_cmd_interpreter_agent.query("This is a list of values that would be sent to the DLP900. What command is being given? `['0x40', '0x0', '0x3', '0x0', '0x1', '0x2', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']`")

Added user message to memory: This is a list of values that would be sent to the DLP900. What command is being given? `['0x40', '0x0', '0x3', '0x0', '0x1', '0x2', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']`
=== Calling Function ===
Calling function: tool_dlpu018j with args: {"input":"0x40 0x0 0x3 0x0 0x1 0x2 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0"}
Added user message to memory: 0x40 0x0 0x3 0x0 0x1 0x2 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x

In [43]:
pprint(response)

Response(response='The command being given is related to the expanded data in '
                  'the Run-Length Encoding (RLE) Compression Example. This '
                  'sequence represents data that has been decompressed from an '
                  'RLE-compressed format. If you have any further questions or '
                  'need more details about this data, feel free to ask!',
         source_nodes=[NodeWithScore(node=TextNode(id_='e7b072e5-b19f-42f7-ad3e-eac7f2b7d122', embedding=None, metadata={'file_path': '../../manuals/dmd/dlpu018j.pdf', 'file_name': 'dlpu018j.pdf', 'file_type': 'application/pdf', 'file_size': 1728714, 'creation_date': '2024-09-01', 'last_modified_date': '2024-09-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelation

In [81]:
for tool in all_tools:
    pprint(tool.metadata)

ToolMetadata(description='This tool is to aid in tasks like the following: '
                         'This document is useful for accomplishing tasks like '
                         'creating and executing batch files to configure '
                         'settings on the DLP LightCrafter 6500 and 9000.',
             name='tool_dlpu018j',
             fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>,
             return_direct=False)
ToolMetadata(description='This tool is to aid in tasks like the following: '
                         'This document is useful for accomplishing tasks like '
                         'creating and executing batch files to configure '
                         'settings on the DLP LightCrafter 6500 and 9000.',
             name='tool_dlpu028d',
             fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>,
             return_direct=False)


In [35]:
int(0x84)

132

In [48]:
int('00000011',2)

3